In [ ]:
!pip install tensorflow==2.15.0

In [ ]:
# 2.15.0 버전이 나와야 함

import tensorflow as tf
print(tf.__version__)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
tf.test.is_gpu_available()

In [ ]:
# 이미지 처리
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
# json 파일 처리
import json

In [ ]:
# 파일 존재 여부 확인용
import os

In [ ]:
# Train 데이터에서 훈련용 데이터와 검증용 데이터 분리
from sklearn.model_selection import train_test_split

In [ ]:
# 모델
from tensorflow import keras

# 변수 목록

In [ ]:
os.chdir('/content/drive/MyDrive/[perst]데이터셋/MachineLearningProject/02_Fashion Type Classification Model')

In [ ]:
part_label_string = "0:Background / 1:Outer / 2:Top / 3:Bottom / 4:Onepiece"

In [ ]:
# 자꾸 폴더 이름 바꾸는거 까먹어서 아예 변수로 추가
male_female = "Male"
train_test = "Train"

In [ ]:
part_index = 2

part_string = ["Background", "Outer", "Top", "Bottom", "Onepiece"]

In [ ]:
ori_data_dir_name = "./Dataset/Musinsa_Dataset/Train-Test/" + male_female + "/" + train_test + "/"

new_data_dir_name = "./Dataset/Musinsa_Dataset/Train-Test_0916/" + male_female + "/" + part_string[part_index] + "/" + train_test + "/"

img_width = 200
img_height = 200

part_width = 100
part_height = 100

read_size = 5000

model_dir_name = './Model/' # 학습 모델이 저장될 디렉토리명

In [ ]:
print("From: " + ori_data_dir_name)
print("To: " + new_data_dir_name)

print(len(os.listdir(ori_data_dir_name + "Image")))
print(len(os.listdir(ori_data_dir_name + "Json")))

dir_file_size = len(os.listdir(ori_data_dir_name + "Json"))

# 배경 제거 모델

In [ ]:
seg_model_path = model_dir_name + "poly_seg_model_0428_02.keras"

# 분할실행용 변수

In [ ]:
print(len(os.listdir(new_data_dir_name + "Image")))

In [ ]:
print(len(os.listdir(new_data_dir_name + "Json")))

In [ ]:
read_point = 14773

In [ ]:
print(read_point)

In [ ]:
save_point = 14161

In [ ]:
print(save_point)

# 데이터 불러오기

In [ ]:
all_img_list = []
all_json_list = []

while len(all_json_list) < read_size:
  # 디렉토리 내 최대 파일까지 읽은 경우
  if read_point > dir_file_size:
    break

  img_path = ori_data_dir_name + "Image/" + str(read_point) + ".jpg"
  label_path = ori_data_dir_name + "Json/" + str(read_point) + ".json"

  # json 파일 로드
  with open(label_path, "r") as loadfile:
    label_file = json.load(loadfile)
  if label_file is None:
    print(str(read_point) + ".json File Load Failed!!")
    break

  if male_female == "Male":
    if label_file['Labeling'] == 'Romantic': # Male에서 Romantic은 안쓴다
      print("File Skip : Romantic")
      read_point = read_point+1
      continue


  # 이미지 파일 로드
  img_file = cv2.imread(img_path)
  if img_file is None:
    print(str(read_point) + ".jpg File Load Failed!!")
    break


  all_img_list.append(img_file)
  all_json_list.append(label_file)


  if len(all_json_list)%500 == 0: # 진행도 확인용
    print(str(len(all_json_list)) + "개 파일 읽기 완료")

  read_point = read_point+1


print()
print("Complete")
print("다음 read point: " + str(read_point))

# 학습을 위해 X 데이터 형식 변경

In [ ]:
all_img_list = np.array(all_img_list)
all_img_list.shape


# 이미지 세그멘테이션

## 마스크 생성

In [ ]:
# 데이터 크기 확인

seg_model = keras.models.load_model(seg_model_path)
print(all_img_list.shape)

print("Complete")

In [ ]:
# 예측 진행
seg_preds = seg_model.predict(all_img_list)
print("Complete")

In [ ]:
# 예측 결과를 마스크로 생성하고 저장
all_mask_list = []

for i in range(0, len(seg_preds)):
    total_mask = seg_preds[i].argmax(axis=-1)
    all_mask_list.append(total_mask)

all_mask_list = np.array(all_mask_list)
print("Complete")

In [ ]:
# 마스크 형태 확인
print(part_label_string)

plt.figure(figsize=(10, 5))

plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(all_img_list[0], cv2.COLOR_BGR2RGB))
plt.title("Original Image")

plt.subplot(1, 2, 2)
plt.imshow(all_mask_list[0])
plt.clim(0, 4)
plt.colorbar(shrink=0.5)
plt.title("Mask Image")

plt.show()

# 이미지의 특정 part만 추출

In [ ]:
part_index in all_mask_list[0]

In [ ]:
part_img_list = []
part_mask_list = []
part_json_list = []

for i in range(0, len(all_mask_list)):
  if (i%500) == 0: # 진행도 확인용
    print(str(i+1) + "번째 이미지 작업중...")

  if part_index not in all_mask_list[i]:
    continue
  else:
    part_img_list.append(all_img_list[i])
    part_json_list.append(all_json_list[i])

    part_mask = np.zeros(all_mask_list[i].shape, dtype=np.uint8)
    part_mask[all_mask_list[i] == part_index] = 1
    part_mask_list.append(part_mask)

print("Complete")

In [ ]:
part_img_list = np.array(part_img_list)
part_mask_list = np.array(part_mask_list)

print(part_img_list.shape)
print(part_mask_list.shape)
print(len(part_json_list))

In [ ]:
# 마스크 형태 확인
plt.imshow(part_mask_list[0])
plt.clim(0, 1)
plt.colorbar(shrink=0.5)
plt.show()

## 렉트 좌표로 자르기

In [ ]:
croped_img_list = []
croped_json_list = []

for i in range(0, len(part_img_list)):

    if (i%500) == 0: # 진행도 확인용
        print(str(i+1) + "번째 이미지 작업중...")

    x1 = img_width
    y1 = img_height
    x2 = 0
    y2 = 0

    for y in range(0, img_height):
        for x in range(0, img_width):
            if part_mask_list[i][y][x] == 1:
                if x < x1:
                    x1 = x
                elif x > x2:
                    x2 = x

                if y < y1:
                    y1 = y
                elif y > y2:
                    y2 = y

    x = x1
    y = y1
    w = x2-x1
    h = y2-y1

    if h < 10 or w < 10:
      continue
    else:
      croped_img = part_img_list[i][y:y+h, x:x+w]

    croped_img_list.append(croped_img)
    croped_json_list.append(part_json_list[i])


print("Complete")

In [ ]:
print(len(croped_img_list))
print(len(croped_json_list))

In [ ]:
# 이미지 크기 조정
for i in range(0, len(croped_img_list)):
  if (i%500) == 0: # 진행도 확인용
        print(str(i+1) + "번째 이미지 작업중...")

  croped_img_list[i] = cv2.resize(croped_img_list[i], dsize=(part_height, part_width), interpolation=cv2.INTER_AREA)

# np.array 형태로 변환
croped_img_list = np.array(croped_img_list)
print("Complete")
print(croped_img_list.shape)

In [ ]:
# 샘플 이미지 표시를 위해 rgb 형식으로 변환
rgb_img = cv2.cvtColor(part_img_list[0], cv2.COLOR_BGR2RGB)
rgb_crop = cv2.cvtColor(croped_img_list[0], cv2.COLOR_BGR2RGB)

In [ ]:
# 렉트좌표로 자른 후 이미지 확인
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(rgb_img)
plt.title("Original Image")

plt.subplot(1, 3, 2)
plt.imshow(part_mask_list[0])
plt.clim(0, 4)
plt.colorbar(shrink=0.5)
plt.title("Mask Image")

plt.subplot(1, 3, 3)
plt.imshow(rgb_crop)
plt.title("Croped Image")

plt.show()

# 가공된 데이터를 폴더에 저장

In [ ]:
i = save_point

for img in croped_img_list:
  if ((i+1)%500) == 0: # 진행도 확인용
    print(str(i+1) + "번째 이미지 저장중...")

  success = cv2.imwrite(new_data_dir_name + "Image/" + str(i) + ".jpg", img)

  if not success:
    print("Image Save Failed!!")
    break

  i = i+1

print("Complete")

In [ ]:
i = save_point

for json_file in croped_json_list:
  if ((i+1)%500) == 0: # 진행도 확인용
    print(str(i+1) + "번째 Json 저장중...")

  json_path = new_data_dir_name + "Json/" + str(i) + ".json"

  with open(json_path, 'w') as outfile:
    json.dump(json_file, outfile)

  if not os.path.isfile(json_path): # 파일 저장에 실패했을 경우
    print("Json Save Failed!!")
    break

  i = i+1

  save_point = i

print("Complete")
print("다음 save point : " + str(save_point))